# Notebook for Individual Experiment Results

### Libraries import

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy import stats
from sklearn.metrics import auc

sys.path.append("../../")

2022-10-22 03:44:24.993986: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
from utils.common import repeat_vector_to_size
from utils.metrics import precision_recall_curve, tpr_fpr_curve, all_metrics_curve
from utils.metrics import homocedasticity_level, shapeness_level
from utils.metrics import accuracy, precision, recall, specificity, f1_score

### Experiment selection

In [3]:
seed = 8128
np.random.seed(seed)
experiment_id = "0013"
root_path = "../../results/Ganomaly_3D/0012_train_healthy/"
for i in sorted(os.listdir(root_path)):
    if experiment_id in i:
        experiment_folder = os.path.join(root_path, i)
experiment_folder

'../../results/Ganomaly_3D/0012_train_healthy/0013_Ganomaly3D-64x64x64x1'

### Quantitative metrics

##### Errors loading

In [19]:
errors_path = os.path.join(experiment_folder, "outputs/errors")
vectors_path = os.path.join(experiment_folder, "outputs/latent_vectors/input_generator")

# Initializing dict for losses of different elements in network
for t in ["encoder", "contextual", "adversarial"]:
    data = "{}_losses".format(t)
    globals()[data] = {}
    for c in ["normal", "abnormal"]:
        globals()[data][c] = {}
        for m in ["train", "val", "test"]:
            globals()[data][c][m] = {}

for t in ["encoder", "contextual", "adversarial"]:
    for m in ["train", "val", "test"]:
        if m == "train":
            if os.path.isfile(os.path.join(errors_path, t, m, "normal.npy")):
                classes = ["normal"] 
            else:
                classes = ["abnormal"]
        else:
            classes = ["normal", "abnormal"]

        for c in classes:
            error_file = os.path.join(errors_path, t, m, c + ".npy")
            if os.path.isfile(error_file):
                data = "{}_losses".format(t)
                errors = np.load(error_file)
                patients_ids_positions = [
                    int(i.split("_")[1].split("-")[1]) for i in sorted(
                        os.listdir(os.path.join(vectors_path, m, c))
                    )
                ]
                assert len(errors) == len(patients_ids_positions)
                for p_id in np.unique(patients_ids_positions):
                    globals()[data][c][m][p_id] = []
                    
                for i, p_id in enumerate(patients_ids_positions):
                    globals()[data][c][m][p_id].append(errors[i])

In [165]:
def get_stats(p_id, data):
    data_np = np.r_[data]
    n = len(data)
    data_max = np.max(data)
    coverage_max = np.count_nonzero(data_np <= data_max) / n
    mean = np.mean(data)
    coverage_mean = np.count_nonzero(data_np <= mean) / n
    std = np.std(data)
    var = np.var(data)
    median = np.median(data)
    coverage_median = np.count_nonzero(data_np <= median) / n
    m2s = mean + 3*std
    coverage_m2s = np.count_nonzero(data_np <= m2s) / n
    mv = mean + var
    coverage_mv = np.count_nonzero(data_np <= mv) / n
    return [
        p_id,
        round(data_max, 3),
        round(coverage_max, 3),
        round(mean, 3),
        round(coverage_mean, 3),
        round(std, 3),
        round(var, 3),
        round(median, 3),
        round(coverage_median, 3),
        round(m2s, 3),
        round(coverage_m2s, 3),
        round(mv, 3),
        round(coverage_mv, 3)
    ]
    
samples = encoder_losses["normal"]["val"]
table = []
columns = ["patient_id","max", "coverage","mean", "coverage", "std", "var", "median", "coverage", "mean+2std", "coverage", "mean+var", "coverage"]
for p_id in samples:
    table.append(get_stats(p_id, samples[p_id]))
pd.DataFrame(table, columns=columns)

,patient_id,max,coverage,mean,coverage,std,var,median,coverage,mean+2std,coverage,mean+var,coverage
0,9,0.232,1.0,0.182,0.500,0.027,0.001,0.182,0.500,0.265,1.0,0.183,0.500
1,25,2.515,1.0,1.553,0.550,0.415,0.172,1.471,0.500,2.797,1.0,1.725,0.625
2,30,1.475,1.0,0.885,0.522,0.342,0.117,0.882,0.522,1.911,1.0,1.002,0.609


In [156]:
data = []
for c in ["normal", "abnormal"]:
    parts = []
    for g in ["train", "val", "test"]:
        samples = encoder_losses[c][g]
        for p_id in samples:
            parts.append(np.r_[contextual_losses[c][g][p_id]])
    data.append(parts)
homocedasticity_level(*data), shapeness_level(*data, seed=seed)

(0.5595238095238095, 0.6190476190476191)

##### Metrics loading

In [45]:
train_metrics = pd.read_csv(os.path.join(experiment_folder, "metrics/train.csv"))
val_metrics = pd.read_csv(os.path.join(experiment_folder, "metrics/val.csv"))
test_metrics = pd.read_csv(os.path.join(experiment_folder, "metrics/test.csv"))
for i in ["train", "val", "test"]:
    metric_file = globals()["{}_metrics".format(i)]
    index = metric_file.shape[0] - 1
    print("{} metrics".format(i))
    print("AUC: {}".format(metric_file.loc[index, "auc"]))
    print("Acc: {}".format(metric_file.loc[index, "accuracy"]))
    print("Pre: {}".format(metric_file.loc[index, "precision"]))
    print("Rec: {}".format(metric_file.loc[index, "recall"]))
    print("Spe: {}".format(metric_file.loc[index, "specificity"]))
    print("F1: {}".format(metric_file.loc[index, "f1_score"]))
    print("="*15)

train metrics
AUC: 0.0
Acc: 0.8559321761131287
Pre: 0.0
Rec: nan
Spe: 0.8559321761131287
F1: 0.0
val metrics
AUC: 0.6015672087669373
Acc: 0.6478873491287231
Pre: 0.7407407164573669
Rec: 0.3174603283405304
Spe: 0.9113923907279968
F1: 0.4444444444444444
test metrics
AUC: 0.8472222685813904
Acc: 0.5874999761581421
Pre: 0.8666666746139526
Rec: 0.2954545319080353
Spe: 0.9444444179534912
F1: 0.4406779661016949


##### Selecting the threshold and calculating the standard metrics

In [166]:
data_table = []
lambda_value = 3
data_columns = ["Partition", "Group", "AUC-ROC", "Threshold", "Acc", "Pre", "Rec", "Spe", "F1", "Homo", "Shape"]

for t in ["encoder", "contextual", "adversarial"]:
    for part in ["val", "test"]:
        data = "{}_losses".format(t)
        if len(globals()[data]["normal"]["train"]) != 0:
            errors = ["normal", "abnormal"]
        elif len(globals()[data]["abnormal"]["train"]) != 0:
            errors = ["abnormal", "normal"]
        y_true = []
        y_pred = []
        for ci, c in enumerate(errors):
            samples = globals()[data][c][part]
            y_pred += [
                np.mean(samples[i]) + lambda_value*np.std(samples[i]) for i in samples
            ]
            y_true += [ci]*len(samples)
        y_true = np.r_[y_true]
        y_pred = np.r_[y_pred]
        tpr, fpr, _ = tpr_fpr_curve(y_true, y_pred)

        if part == "val":
            accs, pres, recs, spes, f1s, thresholds = all_metrics_curve(y_true, y_pred)
            table_metrics = np.concatenate([
                accs.reshape([-1,1]), 
#                 pres.reshape([-1,1]), 
                recs.reshape([-1,1]), 
#                 spes.reshape([-1,1]), 
#                 f1s.reshape([-1,1])
            ], axis=1)
            threshold = thresholds[np.argmax(np.mean(table_metrics, axis=1))]
            # deltas = np.abs(tpr - fpr)
            # threshold = thresholds[np.argmin(deltas[deltas != 0])]
    
        #threshold = 1.174
        y_pred = (y_pred > threshold).astype(np.int64)

        TP = tf.keras.metrics.TruePositives()
        TN = tf.keras.metrics.TrueNegatives()
        FP = tf.keras.metrics.FalsePositives()
        FN = tf.keras.metrics.FalseNegatives()

        TP.update_state(y_true, y_pred)
        TN.update_state(y_true, y_pred)
        FP.update_state(y_true, y_pred)
        FN.update_state(y_true, y_pred)
        
        classes_data = []
        for c in ["normal", "abnormal"]:
            parts = []
            for g in ["train", "val", "test"]:
                samples = globals()[data][c][g]
                for p_id in samples:
                    parts.append(np.r_[samples[p_id]])
            classes_data.append(parts)


        data_table.append([
            part,
            t,
            round(auc(fpr, tpr), 3),
            round(threshold, 3),
            round(accuracy(TP.result().numpy(), TN.result().numpy(), FP.result().numpy(), FN.result().numpy()), 3),
            round(precision(TP.result().numpy(), FP.result().numpy()), 3),
            round(recall(TP.result().numpy(), FN.result().numpy()), 3),
            round(specificity(TN.result().numpy(), FP.result().numpy()), 3),
            round(f1_score(TP.result().numpy(), FP.result().numpy(), FN.result().numpy()), 3),
            round(homocedasticity_level(*classes_data), 3),
            round(shapeness_level(*classes_data, seed=seed), 3)
        ])
pd.DataFrame(data_table, columns=data_columns)

,Partition,Group,AUC-ROC,Threshold,Acc,Pre,Rec,Spe,F1,Homo,Shape
0,val,encoder,0.222,0.265,0.667,0.600,1.0,0.333,0.750,0.452,0.681
1,test,encoder,0.250,0.265,0.500,0.500,1.0,0.000,0.667,0.452,0.681
2,val,contextual,0.667,0.163,1.000,1.000,1.0,1.000,1.000,0.560,0.619
3,test,contextual,0.500,0.163,0.750,0.667,1.0,0.500,0.800,0.560,0.619
4,val,adversarial,0.556,0.456,0.833,0.750,1.0,0.667,0.857,0.531,0.676
5,test,adversarial,-0.000,0.456,0.500,0.500,1.0,0.000,0.667,0.531,0.676


### Qualitative metrics

##### Dist. Properties

In [ ]:
data_table = []
data_columns = ["Group", "Element", "Class", "Min", "Max", "Mean", "Std", "Ske", "Kur", "CDF(x > 0)"]

for g in ["train", "val", "test", "all"]:
    if g == "train":
        classes = ["check"]
    else:
        classes = ["normal", "abnormal"]
    for cl in classes:
        for t in ["encoder", "contextual", "adversarial"]:
            if cl == "check":
                if "train_{}_normal".format(t) in globals().keys():
                    c = "normal"
                else:
                    c = "abnormal"
            else:
                c = cl
            data = globals()["{}_{}_{}".format(g, t, c)]
            m = np.mean(data)
            s = np.std(data)
            data_table.append([
                g,
                t,
                c,
                np.min(data),
                np.max(data),
                m,
                s,
                stats.skew(data),
                stats.kurtosis(data),
                1 - stats.norm(m, s).cdf(0)
            ])
pd.DataFrame(data_table, columns=data_columns)

##### Grouping tests

In [ ]:
data_table = []
data_columns = ["Element", "G1", "G2", "Brow", "Lev", "Chi2 G1 -> G2", "Chi2 G2 -> G1"]

for t in ["encoder", "contextual", "adversarial"]:
    for g1, g2 in [
        ("train", "val"), ("train", "test"), ("train", "all"), 
        ("val", "test"), ("val", "all"),
        ("test", "all")
    ]:
        if "train_{}_normal".format(t) in globals().keys():
            c = "normal"
        else:
            c = "abnormal"
        data1 = globals()["{}_{}_{}".format(g1, t, c)]
        data2 = globals()["{}_{}_{}".format(g2, t, c)]
        if data1.shape[0] > data2.shape[0]:
            sub_data1 = np.r_[sorted(data1)]
            sub_data2 = np.r_[sorted(repeat_vector_to_size(data2, data1.shape[0]))]
        elif data1.shape[0] < data2.shape[0]:
            sub_data1 = np.r_[sorted(repeat_vector_to_size(data1, data2.shape[0]))]
            sub_data2 = np.r_[sorted(data2)]
        else:
            sub_data1 = data1
            sub_data2 = data2
        chi_test_g1 = chiSquare_test(sub_data1, sub_data2)
        chi_test_g2 = chiSquare_test(sub_data2, sub_data1)
        data_table.append([
            t,
            g1,
            g2,
            int(brownForsythe_test(data1, data2)),
            int(levene_test(data1, data2)),
            "{} ({})".format(int(chi_test_g1[0]), round(chi_test_g1[1], 5)),
            "{} ({})".format(int(chi_test_g2[0]), round(chi_test_g2[1], 5)),
        ])
pd.DataFrame(data_table, columns=data_columns)

In [ ]:
data_table = []
data_columns = ["Element", "G1", "G2", "Brow", "Lev", "Chi2 G1 -> G2", "Chi2 G2 -> G1"]

for t in ["encoder", "contextual", "adversarial"]:
    for g1, g2 in [
        ("train", "val"), ("train", "test"), #("train", "all"), 
        ("val", "test"), #("val", "all"),
        #("test", "all")
    ]:
        if "train_{}_normal".format(t) in globals().keys():
            c = "normal"
        else:
            c = "abnormal"
        data1 = globals()["{}_{}_{}".format(g1, t, c)]
        data2 = globals()["{}_{}_{}".format(g2, t, "abnormal")]
        if data1.shape[0] > data2.shape[0]:
            sub_data1 = np.r_[sorted(data1)]
            sub_data2 = np.r_[sorted(repeat_vector_to_size(data2, data1.shape[0]))]
        elif data1.shape[0] < data2.shape[0]:
            sub_data1 = np.r_[sorted(repeat_vector_to_size(data1, data2.shape[0]))]
            sub_data2 = np.r_[sorted(data2)]
        else:
            sub_data1 = data1
            sub_data2 = data2
        chi_test_g1 = chiSquare_test(sub_data1, sub_data2)
        chi_test_g2 = chiSquare_test(sub_data2, sub_data1)
        data_table.append([
            t,
            g1,
            g2,
            int(brownForsythe_test(data1, data2)),
            int(levene_test(data1, data2)),
            "{} ({})".format(int(chi_test_g1[0]), round(chi_test_g1[1], 5)),
            "{} ({})".format(int(chi_test_g2[0]), round(chi_test_g2[1], 5)),
        ])
pd.DataFrame(data_table, columns=data_columns)

### Classing tests

In [ ]:
data_table = []
data_columns = ["Group", "Element", "Brow", "Lev", "Chi2 N -> A", "Chi2 A -> N"]
for t in ["encoder", "contextual", "adversarial"]:
    for g1 in ["train", "val", "test", "all"]:
        for g2 in ["val", "test", "all"]:
            if "train_{}_normal".format(t) in globals().keys():
                data1 = globals()["{}_{}_normal".format(g1, t)]
                data2 = globals()["{}_{}_abnormal".format(g2, t)]
            else:
                data2 = globals()["{}_{}_abnormal".format(g1, t)]
                data1 = globals()["{}_{}_normal".format(g2, t)]
            if data1.shape[0] > data2.shape[0]:
                sub_data1 = np.r_[sorted(data1)]
                sub_data2 = np.r_[sorted(repeat_vector_to_size(data2, data1.shape[0]))]
            elif data1.shape[0] < data2.shape[0]:
                sub_data1 = np.r_[sorted(repeat_vector_to_size(data1, data2.shape[0]))]
                sub_data2 = np.r_[sorted(data2)]
            else:
                sub_data1 = np.r_[sorted(data1)]
                sub_data2 = np.r_[sorted(data2)]
            chi_test_1 = chiSquare_test(sub_data1, sub_data2)
            chi_test_2 = chiSquare_test(sub_data2, sub_data1)
            data_table.append([
                "{} vs {}".format(g1, g2),
                t, 
                int(brownForsythe_test(data1, data2)),
                int(levene_test(data1, data2)),
                "{} ({})".format(int(chi_test_1[0]), round(chi_test_1[1], 5)),
                "{} ({})".format(int(chi_test_2[0]), round(chi_test_2[1], 5))
            ])
pd.DataFrame(data_table, columns=data_columns)